<a href="https://colab.research.google.com/github/MAHESH3368/banking_system-_oop.py/blob/main/Banking_system_oop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from abc import ABC, abstractmethod
from datetime import datetime
import uuid

# ---------------------- TRANSACTION ----------------------
class Transaction:
    def __init__(self, transaction_type, amount):
        self.id = str(uuid.uuid4())[:8]
        self.transaction_type = transaction_type
        self.amount = amount
        self.timestamp = datetime.now()

    def __str__(self):
        return f"[{self.timestamp}] {self.transaction_type}: ${self.amount}"


# ---------------------- ACCOUNT (ABSTRACT) ----------------------
class Account(ABC):
    def __init__(self, account_number, owner):
        self.account_number = account_number
        self.owner = owner
        self._balance = 0.0  # Encapsulation
        self.transactions = []

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Deposit amount must be positive.")
        self._balance += amount
        self.transactions.append(Transaction("Deposit", amount))

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive.")
        if amount > self._balance:
            raise ValueError("Insufficient balance.")
        self._balance -= amount
        self.transactions.append(Transaction("Withdraw", amount))

    def get_balance(self):
        return self._balance

    def show_transactions(self):
        for tx in self.transactions:
            print(tx)

    @abstractmethod
    def calculate_interest(self):
        pass


# ---------------------- SAVINGS ACCOUNT ----------------------
class SavingsAccount(Account):
    INTEREST_RATE = 0.04

    def calculate_interest(self):
        interest = self._balance * self.INTEREST_RATE
        self._balance += interest
        self.transactions.append(Transaction("Interest", interest))


# ---------------------- CURRENT ACCOUNT ----------------------
class CurrentAccount(Account):
    def calculate_interest(self):
        # No interest for current account
        pass


# ---------------------- CUSTOMER ----------------------
class Customer:
    def __init__(self, name, email):
        self.customer_id = str(uuid.uuid4())[:6]
        self.name = name
        self.email = email
        self.accounts = []

    def add_account(self, account):
        self.accounts.append(account)

    def __str__(self):
        return f"{self.name} ({self.customer_id})"


# ---------------------- BANK ----------------------
class Bank:
    def __init__(self, name):
        self.name = name
        self.customers = {}

    def create_customer(self, name, email):
        customer = Customer(name, email)
        self.customers[customer.customer_id] = customer
        return customer

    def create_account(self, customer, account_type):
        account_number = str(uuid.uuid4())[:10]

        if account_type == "savings":
            account = SavingsAccount(account_number, customer)
        elif account_type == "current":
            account = CurrentAccount(account_number, customer)
        else:
            raise ValueError("Invalid account type.")

        customer.add_account(account)
        return account


# ---------------------- MAIN DEMO ----------------------
if __name__ == "__main__":
    bank = Bank("OpenAI Bank")

    # Create customer
    customer = bank.create_customer("John Doe", "john@email.com")

    # Create accounts
    savings = bank.create_account(customer, "savings")

    # Perform operations
    savings.deposit(1000)
    savings.withdraw(200)
    savings.calculate_interest()

    # Output
    print(f"Customer: {customer}")
    print(f"Account Balance: ${savings.get_balance():.2f}")
    print("\nTransaction History:")
    savings.show_transactions()


Customer: John Doe (b2d78a)
Account Balance: $832.00

Transaction History:
[2025-12-31 13:56:11.151760] Deposit: $1000
[2025-12-31 13:56:11.151780] Withdraw: $200
[2025-12-31 13:56:11.151791] Interest: $32.0
